# Legacy XTTS v2 - Local Benchmark

Bu notebook Legacy XTTS v2 engine'ini lokal makinede test eder.

**Engine:** Legacy XTTS v2 (Coqui TTS)
**GPU:** Önerilir (NVIDIA CUDA)
**Türkçe Desteği:** Evet (native multilingual)

**NOT:** Coqui TTS Python 3.12+ ile uyumlu değildir. Bu notebook Miniconda ile Python 3.11 ortamı oluşturarak çalışır.

Her soru için hem SORU hem CEVAP seslendiriliyor.

In [ ]:
# Cell 1: Check Conda & Create Environment
import subprocess
import sys
import os

ENV_NAME = "tts_py311"

# Check if conda is available
try:
    result = subprocess.run(["conda", "--version"], capture_output=True, text=True)
    print(f"Conda found: {result.stdout.strip()}")
except FileNotFoundError:
    print("ERROR: Conda not found!")
    print("Please install Miniconda or Anaconda first:")
    print("  https://docs.conda.io/en/latest/miniconda.html")
    raise SystemExit("Conda required")

# Check if environment exists
result = subprocess.run(["conda", "env", "list"], capture_output=True, text=True)
env_exists = ENV_NAME in result.stdout

if env_exists:
    print(f"Environment '{ENV_NAME}' already exists.")
else:
    print(f"Creating environment '{ENV_NAME}' with Python 3.11...")
    subprocess.run(["conda", "create", "-n", ENV_NAME, "python=3.11", "-c", "conda-forge", "-y"], check=True)
    print("Environment created!")

# Show environments
!conda env list

In [ ]:
# Cell 2: Install Dependencies
import subprocess
import sys
import os

ENV_NAME = "tts_py311"
TTS_MS_PATH = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

def run_in_env(cmd):
    """Run command in conda environment"""
    full_cmd = ["conda", "run", "-n", ENV_NAME] + cmd
    result = subprocess.run(full_cmd, capture_output=True, text=True)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)
    return result.returncode

# Install PyTorch with CUDA
print("Installing PyTorch...")
run_in_env(["pip", "install", "-q", "torch", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu121"])

# Install Coqui TTS (use coqui-tts package, not TTS)
print("\nInstalling Coqui TTS...")
run_in_env(["pip", "install", "-q", "coqui-tts"])

# Install tts-ms from local path
print(f"\nInstalling tts-ms from {TTS_MS_PATH}...")
run_in_env(["pip", "install", "-q", TTS_MS_PATH])

# Verify
print("\n" + "="*60)
print("VERIFICATION:")
run_in_env(["python", "--version"])
run_in_env(["python", "-c", "import torch; print('PyTorch:', torch.__version__, 'CUDA:', torch.cuda.is_available())"])
run_in_env(["python", "-c", "from TTS.api import TTS; print('Coqui TTS: OK')"])
run_in_env(["python", "-c", "import tts_ms; print('tts-ms: OK')"])
print("="*60)

In [ ]:
# Cell 3: Setup Output Directories & Create Benchmark Script
import os
import shutil

ENGINE_NAME = "legacy"
MODEL_NAME = "xtts_v2"

# Use local output directory
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
BASE_DIR = os.path.join(NOTEBOOK_DIR, "output", ENGINE_NAME)
AUDIO_DIR = os.path.join(BASE_DIR, "audio")
MODEL_DIR = os.path.join(NOTEBOOK_DIR, "cache", "xtts")

# Clean start
if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"Output directory: {BASE_DIR}")
print(f"Model cache: {MODEL_DIR}")

# Write the benchmark script
SCRIPT = f'''
import os
import sys
import json
import time
import psutil
from datetime import datetime

# Fix matplotlib backend issue
os.environ["MPLBACKEND"] = "agg"

# Environment setup
os.environ["TTS_MODEL_TYPE"] = "legacy"
os.environ["TTS_MS_LOG_LEVEL"] = "4"
os.environ["TTS_MS_RESOURCES_ENABLED"] = "1"
os.environ["COQUI_TOS_AGREED"] = "1"
os.environ["TTS_HOME"] = r"{MODEL_DIR}"

ENGINE_NAME = "{ENGINE_NAME}"
MODEL_NAME = "{MODEL_NAME}"
BASE_DIR = r"{BASE_DIR}"
AUDIO_DIR = r"{AUDIO_DIR}"

from tts_ms.services import TTSService
from tts_ms.services.tts_service import SynthesizeRequest
from tts_ms.core.config import Settings

process = psutil.Process()
CPU_COUNT = psutil.cpu_count()
resource_logs = []
results = []

def get_resources():
    return {{"cpu": process.cpu_percent(), "ram_mb": process.memory_info().rss / 1024 / 1024}}

QUESTIONS = [
    {{"id": "01", "question": "Sizi neden işe almalıyız?",
     "answer": "Güçlü analitik düşünme becerilerim ve takım çalışmasına yatkınlığım sayesinde projelere değer katabilirim. Ayrıca sürekli öğrenmeye açık yapım ve problem çözme yeteneklerim, şirketinizin hedeflerine ulaşmasında önemli katkılar sağlayacaktır."}},
    {{"id": "02", "question": "Siz bizi neden seçtiniz?",
     "answer": "Şirketinizin yenilikçi yaklaşımı ve sektördeki lider konumu beni çok etkiledi. Kariyer hedeflerimle örtüşen bu ortamda kendimi geliştirebileceğime ve anlamlı projeler üzerinde çalışabileceğime inanıyorum."}},
    {{"id": "03", "question": "Kötü özellikleriniz nelerdir?",
     "answer": "Bazen aşırı detaycı olabiliyorum, bu da zaman yönetimimi olumsuz etkileyebiliyor. Ancak bu özelliğimin farkındayım ve önceliklendirme teknikleri kullanarak bu durumu yönetmeye çalışıyorum."}},
    {{"id": "04", "question": "Beş yıl sonra kendinizi nerede görüyorsunuz?",
     "answer": "Beş yıl içinde teknik liderlik pozisyonunda olmayı hedefliyorum. Ekip yönetimi deneyimi kazanarak şirketin büyümesine stratejik katkılar sağlamak istiyorum."}},
    {{"id": "05", "question": "Maaş beklentiniz nedir?",
     "answer": "Piyasa koşullarını ve pozisyonun gerekliliklerini değerlendirerek, deneyimime ve yeteneklerime uygun rekabetçi bir maaş beklentim var. Bu konuda esnek olmaya ve karşılıklı bir anlaşmaya varmaya açığım."}}
]

print("="*60)
print("INITIALIZING TTS SERVICE")
print("="*60)

res_before = get_resources()
start = time.time()

settings = Settings(raw={{
    "tts": {{
        "engine": "legacy",
        "model_name": "tts_models/multilingual/multi-dataset/xtts_v2",
        "device": "cuda",
        "default_language": "tr",
        "default_speaker": "Ana Florence"
    }},
    "cache": {{"enabled": False}},
    "storage": {{"enabled": False}},
    "logging": {{"level": 4}}
}})
service = TTSService(settings)
init_time = time.time() - start
res_after = get_resources()
resource_logs.append({{"stage": "init", "duration": init_time, "cpu": res_after["cpu"], "ram_delta": res_after["ram_mb"] - res_before["ram_mb"]}})
print(f"Initialized in {{init_time:.2f}}s")

# Warmup
print("\\nWarmup...")
res_before = get_resources()
start = time.time()
_ = service.synthesize(SynthesizeRequest(text="Merhaba."), request_id="warmup")
warmup_time = time.time() - start
res_after = get_resources()
resource_logs.append({{"stage": "warmup", "duration": warmup_time, "cpu": res_after["cpu"], "ram_delta": res_after["ram_mb"] - res_before["ram_mb"]}})
print(f"Warmup done in {{warmup_time:.2f}}s")

print("\\n" + "="*60)
print(f"SYNTHESIZING {{len(QUESTIONS)}} QUESTIONS + ANSWERS")
print("="*60)

for q in QUESTIONS:
    print(f"\\n[{{q['id']}}] {{q['question']}}")
    
    for typ, text in [("soru", q["question"]), ("cevap", q["answer"])]:
        print(f"  {{typ.upper()}}: ", end="", flush=True)
        res_before = get_resources()
        start = time.time()
        try:
            result = service.synthesize(SynthesizeRequest(text=text), request_id=f"{{q['id']}}_{{typ}}")
            elapsed = time.time() - start
            res_after = get_resources()
            
            path = os.path.join(AUDIO_DIR, f"{{q['id']}}_{{typ}}.wav")
            with open(path, "wb") as f:
                f.write(result.wav_bytes)
            
            cpu_norm = res_after["cpu"] / CPU_COUNT
            ram_delta = res_after["ram_mb"] - res_before["ram_mb"]
            
            resource_logs.append({{"stage": f"{{q['id']}}_{{typ}}", "duration": elapsed, "cpu": res_after["cpu"], "cpu_norm": cpu_norm, "ram_delta": ram_delta, "size_kb": len(result.wav_bytes)/1024}})
            results.append({{"id": q["id"], "type": typ, "time": elapsed, "size": len(result.wav_bytes), "cpu": cpu_norm, "ram_delta": ram_delta, "status": "OK"}})
            
            print(f"{{elapsed:.2f}}s | {{len(result.wav_bytes)/1024:.1f}} KB | CPU:{{cpu_norm:.0f}}% | OK")
        except Exception as e:
            results.append({{"id": q["id"], "type": typ, "time": time.time()-start, "size": 0, "cpu": 0, "ram_delta": 0, "status": f"FAIL: {{e}}"}})
            print(f"FAIL: {{e}}")

successful = [r for r in results if r["status"] == "OK"]
print(f"\\n" + "="*60)
print(f"COMPLETE: {{len(successful)}}/{{len(results)}} successful")
print("="*60)

# Save results
output_data = {{
    "engine": ENGINE_NAME,
    "model": MODEL_NAME,
    "init_time": init_time,
    "warmup_time": warmup_time,
    "results": results,
    "resource_logs": resource_logs,
    "timestamp": datetime.now().isoformat()
}}

with open(os.path.join(BASE_DIR, "results.json"), "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\\nResults saved to {{BASE_DIR}}")
'''

script_path = os.path.join(NOTEBOOK_DIR, "benchmark_script.py")
with open(script_path, "w", encoding="utf-8") as f:
    f.write(SCRIPT)

print(f"Benchmark script created: {script_path}")

In [ ]:
# Cell 4: Run Benchmark
import subprocess
import os

ENV_NAME = "tts_py311"
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
SCRIPT_PATH = os.path.join(NOTEBOOK_DIR, "benchmark_script.py")

print("Running benchmark in Python 3.11 environment...")
print("This may take a while on first run (model download ~1.8GB)\n")

# Set MPLBACKEND environment variable and run
env = os.environ.copy()
env["MPLBACKEND"] = "agg"

result = subprocess.run(
    ["conda", "run", "-n", ENV_NAME, "python", SCRIPT_PATH],
    env=env,
    capture_output=False
)

if result.returncode != 0:
    print(f"Benchmark failed with return code {result.returncode}")

In [ ]:
# Cell 5: Play Audio Files
from IPython.display import Audio, display
from pathlib import Path
import os

ENGINE_NAME = "legacy"
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
AUDIO_DIR = os.path.join(NOTEBOOK_DIR, "output", ENGINE_NAME, "audio")

print("="*60)
print("AUDIO PLAYBACK")
print("="*60)

audio_files = sorted(Path(AUDIO_DIR).glob("*.wav"))

if not audio_files:
    print("No audio files found!")
else:
    for wav in audio_files:
        print(f"\n{wav.name}:")
        display(Audio(str(wav)))

In [ ]:
# Cell 6: Generate Reports
import json
import os
from datetime import datetime
import psutil

ENGINE_NAME = "legacy"
MODEL_NAME = "xtts_v2"
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
BASE_DIR = os.path.join(NOTEBOOK_DIR, "output", ENGINE_NAME)
CPU_COUNT = psutil.cpu_count()

# Load results
with open(os.path.join(BASE_DIR, "results.json"), "r", encoding="utf-8") as f:
    data = json.load(f)

results = data["results"]
resource_logs = data["resource_logs"]
init_time = data["init_time"]
warmup_time = data["warmup_time"]

successful = [r for r in results if r["status"] == "OK"]
soru_results = [r for r in successful if r["type"] == "soru"]
cevap_results = [r for r in successful if r["type"] == "cevap"]

total_size_kb = sum(r["size"] for r in successful) / 1024
avg_soru = sum(r["time"] for r in soru_results) / len(soru_results) if soru_results else 0
avg_cevap = sum(r["time"] for r in cevap_results) / len(cevap_results) if cevap_results else 0

synth_logs = [r for r in resource_logs if r["stage"] not in ["init", "warmup"]]
avg_cpu = sum(r.get("cpu_norm", 0) for r in synth_logs) / len(synth_logs) if synth_logs else 0
max_cpu = max(r.get("cpu_norm", 0) for r in synth_logs) if synth_logs else 0
total_ram = sum(r.get("ram_delta", 0) for r in synth_logs)

# summary.txt
summary = f"""============================================================
TTS-MS BENCHMARK - LEGACY XTTS v2 (LOCAL)
============================================================
Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Platform: Local (Miniconda Python 3.11)
Model: {MODEL_NAME}

PERFORMANS:
- Init: {init_time:.2f}s
- Warmup: {warmup_time:.2f}s

SONUCLAR:
- Toplam: 5 soru
- Soru Audio: {len(soru_results)}/5 basarili (ort: {avg_soru:.2f}s)
- Cevap Audio: {len(cevap_results)}/5 basarili (ort: {avg_cevap:.2f}s)
- Toplam Audio: {total_size_kb:.1f} KB

KAYNAK KULLANIMI:
- Ortalama CPU: {avg_cpu:.0f}%
- Maksimum CPU: {max_cpu:.0f}%
- RAM Delta: {total_ram:+.1f} MB

DETAYLAR:
"""
for r in results:
    s = "OK" if r["status"] == "OK" else "FAIL"
    summary += f"[{r['id']}_{r['type']}] {r['time']:.2f}s | {r['size']/1024:.1f} KB | CPU:{r['cpu']:.0f}% | {s}\n"
summary += f"\nAudio: {os.path.join(BASE_DIR, 'audio')}\n============================================================\n"

with open(os.path.join(BASE_DIR, "summary.txt"), "w", encoding="utf-8") as f:
    f.write(summary)

print(summary)

In [ ]:
# Cell 7: Create ZIP (Optional)
import zipfile
from pathlib import Path
import os

ENGINE_NAME = "legacy"
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
BASE_DIR = os.path.join(NOTEBOOK_DIR, "output", ENGINE_NAME)
AUDIO_DIR = os.path.join(BASE_DIR, "audio")

zip_path = os.path.join(BASE_DIR, "legacy_benchmark.zip")

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    zf.write(os.path.join(BASE_DIR, "summary.txt"), "summary.txt")
    zf.write(os.path.join(BASE_DIR, "results.json"), "results.json")
    for wav in Path(AUDIO_DIR).glob("*.wav"):
        zf.write(wav, f"audio/{wav.name}")

print(f"ZIP created: {zip_path}")
print(f"Size: {Path(zip_path).stat().st_size/1024:.1f} KB")
print("\nContents:")
with zipfile.ZipFile(zip_path, "r") as zf:
    for f in zf.namelist():
        print(f"  {f}")